In [1]:
import pandas as pd
import numpy as np 
import os
import csv
import altair as alt
import statsmodels.formula.api as smf
import statsmodels.api as sm

anoi = 2016 # especificar o ano inicial a ser estudado
anof = 2024 # especificar o ano final a ser estudado
anos = []
for i in range(anof-anoi+1): # cria uma lista dos anos a serem estudados 
    anos.append(anof-(i))


df = pd.DataFrame()

In [2]:
#Base de dados utilizados foram: 
# 1) A base de dados de preços de tarifas aéreas da Anac, disponivél em: https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/tarifas-aereas-domesticas
# 2) A base de dados de preços de distribuição de combustivél da Anp, disponível em: https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis
#Objetivo do trabalho é analisar o que impacta o preço das passagens aéreas domésticas na rota RJ-SP

In [3]:
#Abre todos os arquivos e monta o data frame 

for ano in anos:

    pastadados = f'anac_{ano}'
    nome_arquivos = os.listdir(pastadados)

    Primeiro_Lido = False
    for i in nome_arquivos: 

        with open(pastadados+'/'+str(i), mode = 'r', encoding = 'latin-1') as arq: #descobre o separador do arquivo
            sniff = (csv.Sniffer().sniff(arq.readline())).delimiter

        if not Primeiro_Lido:
            arquivo = pd.read_csv(pastadados+'/'+str(i),sep = sniff, encoding='latin-1', decimal=',', low_memory=False) # Lê o primeiro
            arquivo.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'] #padroniza os arquivos para o mesmo cabeçalho
            Primeiro_Lido = True

        else:
            arquivo_corrido = pd.read_csv(pastadados+'/'+str(i),sep = sniff, encoding='latin-1', decimal=',',low_memory=False) 
        

            while len(arquivo_corrido.columns)>7:
                del arquivo_corrido[f'{arquivo_corrido.columns[0]}'] #retira colunas vazias ou de contagem de linhas

        
                

            arquivo_corrido.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'] #padroniza os arquivos para o mesmo cabeçalho
            arquivo = pd.concat([arquivo, arquivo_corrido]) 

    df = pd.concat([df, arquivo]) 
    

df['TARIFA'] = pd.to_numeric(df['TARIFA'], errors='coerce') # define a coluna tarifa como número e não objeto


df['RECEITA'] = df['TARIFA']*df['ASSENTOS'] #cria coluna de receita


aeroportos_sp_icao = ['SBSP', 'SBKP']
aeroportos_rj_icao = ['SBRJ', 'SBGL']

#filtra a rota RJ-SP, principal rota da aviação comercial (rota trabalhada no projeto)

tabela_filtrada = df[df['ORIGEM'].isin(aeroportos_rj_icao) & df['DESTINO'].isin(aeroportos_sp_icao)]
tabela_filtrado2= df[df['ORIGEM'].isin(aeroportos_sp_icao) & df['DESTINO'].isin(aeroportos_rj_icao)]

dfrota = pd.concat([tabela_filtrada, tabela_filtrado2]) 

#Data Frame principal a ser utilizado
dfrota

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,RECEITA
79483,2024,1,AZU,SBGL,SBKP,159.90,23,3677.70
79484,2024,1,AZU,SBGL,SBKP,163.69,1,163.69
79485,2024,1,AZU,SBGL,SBKP,163.70,43,7039.10
79486,2024,1,AZU,SBGL,SBKP,163.72,3,491.16
79487,2024,1,AZU,SBGL,SBKP,179.90,3,539.70
...,...,...,...,...,...,...,...,...
326682,2016,12,TAM,SBSP,SBRJ,748.02,1,748.02
326871,2016,12,TAM,SBSP,SBRJ,804.92,3,2414.76
326974,2016,12,TAM,SBSP,SBRJ,123.98,2,247.96
327011,2016,12,TAM,SBSP,SBRJ,1325.86,1,1325.86


In [4]:
#----------------------------Gráfico 1: Tarifa Média Anual ----------------------------------------------

alt.renderers.enable('html') #Não permite que crie um html apos executar o codigo

# monta o df_1 - (Tarifa Media Mensal Por Ano)

df_1 = dfrota.groupby(['ANO', 'MES'])[['RECEITA', 'ASSENTOS']].sum()
df_1['TARIFA_MEDIA_MENSAL'] = df_1['RECEITA'] / df_1['ASSENTOS']
df_1 = df_1.reset_index()[['ANO', 'MES', 'TARIFA_MEDIA_MENSAL']]

df_1['ANO'] = df_1['ANO'].astype(int)
df_1['MES'] = df_1['MES'].astype(int)   

# Fazendo com o que o usuário consiga interragir com o gráfico 

hover = alt.selection(
    type='single',
    fields=['ANO'],
    on='mouseover',
    nearest=True,
    empty='none'
)


pontos = (
    alt.Chart(df_1)
    .mark_circle(size=100, opacity=0)
    .encode(
        x=alt.X('MES:O', sort=list(range(1,13))),
        y='TARIFA_MEDIA_MENSAL:Q',
        tooltip=['ANO', 'MES', 'TARIFA_MEDIA_MENSAL'],
        
    )
    .add_params(hover)
)


linhas = (
    alt.Chart(df_1)
    .mark_line(point=True)
    .encode(
        x=alt.X('MES:O', title='Mês', sort=list(range(1,13)), axis=alt.Axis(grid=False)),  # <-- ORDEM CERTA
        y=alt.Y('TARIFA_MEDIA_MENSAL:Q', title='(R$)', axis=alt.Axis(grid=False)
),
        detail='ANO:N',
        color=alt.condition(hover, alt.value('blue'), alt.value('lightgray')),
        size=alt.condition(hover, alt.value(4), alt.value(1))
    )
)

grafico = (linhas + pontos).properties(
    width=650,
    height=350,
    title='Tarifa Média  Mensal Por Ano'
).interactive()

grafico

C:\Users\vitor\AppData\Local\Temp\ipykernel_30704\1913305463.py:16: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use 'selection_point()' or 'selection_interval()' instead.
These functions also include more helpful docstrings.
  hover = alt.selection(


alt.LayerChart(...)

In [5]:
'''Neste primeiro gráfico, observamos a evolução da tarifa média mensal ao longo dos anos para a rota Rio-São Paulo. 
É possível notar dois movimentos principais:

Tendência de Alta: Um aumento progressivo no patamar de preços, especialmente perceptível a partir de 2021/2022, pós-pandemia.

Volatilidade Mensal: A existência de picos e vales dentro de um mesmo ano, sugerindo uma forte componente 
sazonal na precificação das passagens.'''

'Neste primeiro gráfico, observamos a evolução da tarifa média mensal ao longo dos anos para a rota Rio-São Paulo. \nÉ possível notar dois movimentos principais:\n\nTendência de Alta: Um aumento progressivo no patamar de preços, especialmente perceptível a partir de 2021/2022, pós-pandemia.\n\nVolatilidade Mensal: A existência de picos e vales dentro de um mesmo ano, sugerindo uma forte componente \nsazonal na precificação das passagens.'

In [6]:
#Criando um Dataframe para facilitar quando fizermos um mapa de calor 

df_tma = dfrota.groupby('ANO').agg(
    RECEITA_TOTAL_ANUAL=('RECEITA', 'sum'),
    ASSENTOS_TOTAL_ANUAL=('ASSENTOS', 'sum')
).reset_index()

# Calculando a Tarifa Média Anual (TMA)

df_tma['TARIFA_MEDIA_ANUAL'] = df_tma['RECEITA_TOTAL_ANUAL'] / df_tma['ASSENTOS_TOTAL_ANUAL']


df_tma = df_tma[['ANO', 'TARIFA_MEDIA_ANUAL']]


dfrota_com_tma = pd.merge(
    dfrota,
    df_tma,
    on='ANO',
    how='left' 
)

df_tma_resumo = dfrota_com_tma.groupby('ANO')['TARIFA_MEDIA_ANUAL'].first().reset_index()

df_tma_mensal = pd.merge(
    df_1,
    df_tma_resumo,
    on='ANO',
    how='left' # Usamos 'left' para garantir que todas as linhas de df_1 sejam mantidas
)

df_tma_mensal['VARIACAO'] = df_tma_mensal['TARIFA_MEDIA_MENSAL']/df_tma_mensal['TARIFA_MEDIA_ANUAL'] - 1

df_tma_mensal

,ANO,MES,TARIFA_MEDIA_MENSAL,TARIFA_MEDIA_ANUAL,VARIACAO
0,2016,1,248.006455,264.678719,-0.062991
1,2016,2,295.685963,264.678719,0.117150
2,2016,3,301.246760,264.678719,0.138160
3,2016,4,272.699194,264.678719,0.030303
4,2016,5,239.026464,264.678719,-0.096918
...,...,...,...,...,...
103,2024,8,416.232475,448.663894,-0.072284
104,2024,9,451.529319,448.663894,0.006387
105,2024,10,515.465986,448.663894,0.148891
106,2024,11,509.251856,448.663894,0.135041


In [7]:
#-------------------------Gráfico 2: Mapa de Calor para analisar a sazonalidade -------------------------------------------------------

df_heatmap = df_tma_mensal.pivot_table(
    index='ANO', 
    columns='MES', 
    values='VARIACAO'
)

df_heatmap_long = df_heatmap.reset_index().melt(
    id_vars=['ANO'], 
    var_name='MES', 
    value_name='VARIACAO'
).dropna()

heatmap = alt.Chart(df_heatmap_long).encode(
    # Eixo X: Meses (Coluna), Ordenado numericamente
    x=alt.X('MES:O', title='Mês'),
    
    # Eixo Y: Anos (Linha)
    y=alt.Y('ANO:O', title='Ano'),
    
    # Cor: Intensidade da Tarifa
    color=alt.Color('VARIACAO:Q', 
        scale=alt.Scale(domain=[df_heatmap_long['VARIACAO'].min(), 0, df_heatmap_long['VARIACAO'].max()],
        range=['blue', 'white', 'red']), 
        legend=alt.Legend(title="Variação Relativa da Tarifa")
    ),
    
    tooltip=[
        alt.Tooltip('ANO:O'),
        alt.Tooltip('MES:O'),
        alt.Tooltip('VARIACAO:Q', title='Variação Relativa da Tarifa', format='.2f')
    ]
).properties(
    title='Anomalias da Tarifa (Mensal vs. Anual)' ,width=600,
    height=400
)

text = heatmap.mark_text().encode(
    text=alt.Text('VARIACAO:Q', format='.0f'), 
    color=alt.value('black'))

grafico_final_heatmap = (heatmap.mark_rect()).interactive()

grafico_final_heatmap


alt.Chart(...)

In [8]:
'''
Para isolar o efeito da inflação e analisar puramente a sazonalidade, este mapa de calor compara a tarifa de cada mês
com a média daquele ano específico.

Cores Vermelhas (Variação Positiva): Indicam meses onde viajar é mais caro que a média anual.
Nota-se uma concentração nos meses de férias (dezembro/janeiro) e, em alguns anos, no meio do ano.

Cores Azuis (Variação Negativa): Indicam os meses de baixa temporada, onde as tarifas tendem a ficar abaixo da média anual.
'''

'\nPara isolar o efeito da inflação e analisar puramente a sazonalidade, este mapa de calor compara a tarifa de cada mês\ncom a média daquele ano específico.\n\nCores Vermelhas (Variação Positiva): Indicam meses onde viajar é mais caro que a média anual.\nNota-se uma concentração nos meses de férias (dezembro/janeiro) e, em alguns anos, no meio do ano.\n\nCores Azuis (Variação Negativa): Indicam os meses de baixa temporada, onde as tarifas tendem a ficar abaixo da média anual.\n'

In [9]:
#Quantidade de assentos vendidos por mes,
df_plot = dfrota.groupby(['ANO', 'MES'])['ASSENTOS'].sum().reset_index()

grafico = (
    alt.Chart(df_plot)
    .mark_bar()
    .encode(
        x=alt.X('MES:O', title='Mês', sort=list(range(1,13))),  # garante jan→dez
        y=alt.Y('ASSENTOS:Q', title='Assentos vendidos', axis=alt.Axis(grid=False)),
        color='ANO:N',
        tooltip=['ANO', 'MES', 'ASSENTOS']
    )
    .facet(
        facet='ANO:N',
        columns=3,                     # <<<<< número de colunas
        title='Assentos Vendidos por Ano'
    )
    .resolve_scale(y='independent')    # Cada painel com escala própria (opcional)
)

grafico




alt.FacetChart(...)

In [10]:
'''
Aqui analisamos a demanda através do volume de assentos vendidos. O impacto da pandemia de COVID-19 é visualmente drástico em 2020,
com uma queda abrupta a partir de março. A recuperação é gradual nos anos seguintes (2021-2022), 
mas nota-se que o volume de passageiros ainda oscila, não tendo retornado imediatamente aos patamares de estabilidade vistos em 2016-2019.
'''

'\nAqui analisamos a demanda através do volume de assentos vendidos. O impacto da pandemia de COVID-19 é visualmente drástico em 2020,\ncom uma queda abrupta a partir de março. A recuperação é gradual nos anos seguintes (2021-2022), \nmas nota-se que o volume de passageiros ainda oscila, não tendo retornado imediatamente aos patamares de estabilidade vistos em 2016-2019.\n'

In [11]:
# Criar df mensal (RECEITA, ASSENTOS e TARIFA MÉDIA)
df_plot = dfrota.groupby(['ANO', 'MES'])[['RECEITA', 'ASSENTOS']].sum().reset_index()
df_plot['TARIFA_MEDIA_MENSAL'] = df_plot['RECEITA'] / df_plot['ASSENTOS']


# 2) Calcular correlação

corr = df_plot['TARIFA_MEDIA_MENSAL'].corr(df_plot['ASSENTOS'])


# 4) Gráfico scatter 

dispersao_tarifa_assentos = (
    alt.Chart(df_plot)
    .mark_circle(size=80, opacity=0.6)
    .encode(
        x=alt.X('TARIFA_MEDIA_MENSAL:Q', title='Tarifa Média (R$)', axis=alt.Axis(grid=False)),
        y=alt.Y('ASSENTOS:Q', title='Assentos Vendidos',axis=alt.Axis(grid=False)),
        color='ANO:N',
        tooltip=['ANO', 'MES', 'TARIFA_MEDIA_MENSAL', 'ASSENTOS']
    )
)


dispersao_tarifa_assentos_final = (dispersao_tarifa_assentos).properties(
    title=f'Correlação entre Assentos vendidos e Tarida Aérea Média (Correlação: {corr:.2f})',
    width=600,
    height=400
)



dispersao_tarifa_assentos_final


alt.Chart(...)

In [12]:
# Filtrar fora 2020 e 2021
df_corr = df_plot[~df_plot['ANO'].isin([2020, 2021])]

# Calcular correlação sem 2020 e 2021
corr20e21 = df_corr['TARIFA_MEDIA_MENSAL'].corr(df_corr['ASSENTOS'])


dispersao_tarifa_assentos = (
    alt.Chart(df_plot)
    .transform_filter(
        ~alt.FieldEqualPredicate(field='ANO', equal=2020)  # remove 2020
    )
    .transform_filter(
        ~alt.FieldEqualPredicate(field='ANO', equal=2021)  # remove 2021
    )
    .mark_circle(size=80, opacity=0.6)
    .encode(
        x=alt.X('TARIFA_MEDIA_MENSAL:Q', title='Tarifa Média (R$)', axis=alt.Axis(grid=False)),
        y=alt.Y('ASSENTOS:Q', title='Assentos Vendidos', axis=alt.Axis(grid=False)),
        color='ANO:N',
        tooltip=['ANO', 'MES', 'TARIFA_MEDIA_MENSAL', 'ASSENTOS']
    )
)


# Finalizar gráfico
dispersao_tarifa_assentos_sem20e21 = (
    dispersao_tarifa_assentos
    .properties(
        title=f'Correlação entre Assentos vendidos e Tarifa Aérea Média (Sem 2020 e 2021) (Corr = {corr20e21:.2f})',
        width=600,
        height=400
    )
)


dispersao_tarifa_assentos_sem20e21 



alt.Chart(...)

In [13]:
'''
Este gráfico de dispersão testa a lei da oferta e da demanda. A correlação encontrada foi de -0.31.

O valor negativo indica que existe uma tendência de que, quanto maior a tarifa, menor o número de assentos vendidos (e vice-versa).

No entanto, a correlação é considerada fraca/moderada. Porém, retirando-se os outliers da pandemia conseguimos obersar uma relação muio mais forte, com 
correlação de -0,86. Ou seja, conseguimos relacionar fortemente o preço das tarifas aéreas com a quantidade de assentos caso não haja variáveis exógenas 
relevantes.
'''

'\nEste gráfico de dispersão testa a lei da oferta e da demanda. A correlação encontrada foi de -0.31.\n\nO valor negativo indica que existe uma tendência de que, quanto maior a tarifa, menor o número de assentos vendidos (e vice-versa).\n\nNo entanto, a correlação é considerada fraca/moderada. Porém, retirando-se os outliers da pandemia conseguimos obersar uma relação muio mais forte, com \ncorrelação de -0,86. Ou seja, conseguimos relacionar fortemente o preço das tarifas aéreas com a quantidade de assentos caso não haja variáveis exógenas \nrelevantes.\n'

In [14]:
#https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis

#Criando o Data Frame do Preço do Querosene

#Arrumando o dataset
combu = pd.read_excel('combustiveis-aviacao-regioes.xlsx')

combustiveis = combu[combu['REGIÃO'] == 'SUDESTE']

querosene = combustiveis[combustiveis['PRODUTO'] == 'QUEROSENE DE AVIAÇÃO']

querosene['MÊS'] = pd.to_datetime(querosene['MÊS']) #Formata a coluna da data, seperando em ano e mes 
querosene['ANO'] = querosene['MÊS'].dt.year
querosene['MES'] = querosene['MÊS'].dt.month

qav = querosene[['ANO', 'MES', 'UNIDADE DE MEDIDA', 'PREÇO MÉDIO DE DISTRIBUIÇÃO']]

qav

C:\Users\vitor\AppData\Local\Temp\ipykernel_30704\1259318297.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  querosene['MÊS'] = pd.to_datetime(querosene['MÊS']) #Formata a coluna da data, seperando em ano e mes
C:\Users\vitor\AppData\Local\Temp\ipykernel_30704\1259318297.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  querosene['ANO'] = querosene['MÊS'].dt.year
C:\Users\vitor\AppData\Local\Temp\ipykernel_30704\1259318297.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

,ANO,MES,UNIDADE DE MEDIDA,PREÇO MÉDIO DE DISTRIBUIÇÃO
581,2018,11,R$/l,3.492233
582,2018,12,R$/l,3.220913
583,2019,1,R$/l,2.903441
584,2019,2,R$/l,2.841723
585,2019,3,R$/l,2.942767
...,...,...,...,...
659,2025,5,R$/l,4.471988
660,2025,6,R$/l,4.147937
661,2025,7,R$/l,4.281472
662,2025,8,R$/l,4.461266


In [15]:
#--------------------------------- Gráfico 3: Relação do Preço do Querosene e das Tarifas aéreas -------------------------------

#Montando o gráfico de tárifa aérea com relação ao preço do QAV

qav_selecionado = qav[['ANO', 'MES', 'PREÇO MÉDIO DE DISTRIBUIÇÃO']].copy()
qav_selecionado = qav_selecionado.rename(columns={'PREÇO MÉDIO DE DISTRIBUIÇÃO': 'PRECO_QAV'})

df_tarifa_selecionado = df_1[['ANO', 'MES', 'TARIFA_MEDIA_MENSAL']].copy()

# Junção dos DataFrames: Criando o df_combinado

df_combinado = pd.merge(
    df_tarifa_selecionado, 
    qav_selecionado, 
    on=['ANO', 'MES'], 
    how='inner'
)

df_combinado['DATE'] = df_combinado['ANO'].astype(str) + '-' + df_combinado['MES'].astype(str).str.zfill(2)

df_combinado['DATE'] = pd.to_datetime(df_combinado['DATE'])

line_tarifa = alt.Chart(df_combinado).mark_line(color='#1f77b4').encode(
    x=alt.X(
        'DATE:T',
        title='Ano-Mês',
        axis=alt.Axis(
            grid=False,
            format='%Y-%m',
            tickCount=alt.TickCount(interval='month', step=3), 
            labelAngle=-45
        )
    ),
    y=alt.Y(
        'TARIFA_MEDIA_MENSAL:Q',
        axis=alt.Axis(title='Tarifa Aérea Média (R$)', titleColor='#1f77b4', grid=False)
    ),
    tooltip=[
        alt.Tooltip('DATE:T', title='Período', format='%Y-%m'),
        alt.Tooltip('TARIFA_MEDIA_MENSAL', title='Tarifa (R$)', format='.2f')
    ]
)

line_combustivel = alt.Chart(df_combinado).mark_line(color='#ff7f0e').encode(
    x=alt.X(
        'DATE:T',
        axis=alt.Axis(grid=False)
    ),
    y=alt.Y(
        'PRECO_QAV:Q',
        axis=alt.Axis(title='Preço do QAV (R$/L)', titleColor='#ff7f0e', grid=False)
    ),
    tooltip=[
        alt.Tooltip('DATE:T', title='Período', format='%Y-%m'),
        alt.Tooltip('PRECO_QAV', title='QAV (R$/L)', format='.4f')
    ]
)

grafico_final_duplo = (
    line_tarifa + line_combustivel
).resolve_scale(
    y='independent'
).properties(
    title='Tarifa Aérea Média vs. Preço Médio do Querosene de Aviação (QAV)',
    width=1000,
    height=400
).interactive()

grafico_final_duplo


alt.LayerChart(...)

In [16]:
correlacao = df_combinado['PRECO_QAV'].corr(df_combinado['TARIFA_MEDIA_MENSAL'])

dispersao_QAV = alt.Chart(df_combinado).mark_circle(size=60, opacity=0.5).encode(
    x=alt.X('PRECO_QAV:Q', title='Preço do QAV (R$/L)', axis=alt.Axis(grid=False)),
    y=alt.Y('TARIFA_MEDIA_MENSAL:Q', title='Tarifa Aérea Média (R$)', axis=alt.Axis(grid=False)),
    tooltip=[
        alt.Tooltip('DATE:T', title='Período', format='%Y-%m'),
        alt.Tooltip('PRECO_QAV', title='QAV (R$/L)', format='.4f'),
        alt.Tooltip('TARIFA_MEDIA_MENSAL', title='Tarifa (R$)', format='.2f')
    ]
)

# Adicionar a linha de regressão
regression_line = dispersao_QAV.transform_regression(
    'PRECO_QAV', 'TARIFA_MEDIA_MENSAL', method='linear'
).mark_line(color='red')

# Combinar o scatter plot com a linha de regressão
dispersao_QAV_final = (dispersao_QAV + regression_line).properties(
    title=f'Correlação entre Preço do QAV e Tarifa Aérea Média (Correlação: {correlacao:.2f})',
    width=600,
    height=400
)

dispersao_QAV_final

alt.LayerChart(...)

In [17]:
'''
O Querosene de Aviação (QAV) representa um dos maiores custos operacionais das companhias aéreas.

O gráfico de linhas mostra que a curva da tarifa aérea acompanha muito de perto a curva de preço do combustível.

O gráfico de dispersão confirma essa hipótese com uma correlação forte de 0.82.
Isso indica que o aumento dos custos operacionais (combustível) é o principal motor para o aumento das passagens aéreas
no período analisado,
tendo um peso muito maior na precificação do que a simples variação da demanda.
'''

'\nO Querosene de Aviação (QAV) representa um dos maiores custos operacionais das companhias aéreas.\n\nO gráfico de linhas mostra que a curva da tarifa aérea acompanha muito de perto a curva de preço do combustível.\n\nO gráfico de dispersão confirma essa hipótese com uma correlação forte de 0.82.\nIsso indica que o aumento dos custos operacionais (combustível) é o principal motor para o aumento das passagens aéreas\nno período analisado,\ntendo um peso muito maior na precificação do que a simples variação da demanda.\n'

In [18]:


# Criando o DataFrame do Marketshare das empresas de acordo com o ano 
df_ms = (
    dfrota
    .groupby(['ANO', 'EMPRESA'])['ASSENTOS']
    .sum()
    .reset_index()
)

# Market share dentro de cada ano
df_ms['PERCENTUAL'] = (
    df_ms.groupby('ANO')['ASSENTOS']
    .transform(lambda x: x / x.sum())
)

# Criar coluna percentual formatada
df_ms['PERCENTAGEM'] = (df_ms['PERCENTUAL'] * 100).round(2).astype(str) + '%'

df_ms 

,ANO,EMPRESA,ASSENTOS,PERCENTUAL,PERCENTAGEM
0,2016,AZU,308443,0.131838,13.18%
1,2016,GLO,771457,0.329744,32.97%
2,2016,ONE,638668,0.272986,27.3%
3,2016,TAM,620996,0.265432,26.54%
4,2017,AZU,280411,0.131216,13.12%
5,2017,GLO,793991,0.371540,37.15%
6,2017,ONE,516472,0.241678,24.17%
7,2017,TAM,546151,0.255566,25.56%
8,2018,AZU,197061,0.101125,10.11%
9,2018,GLO,839398,0.430752,43.08%


In [19]:


# Ordem fixa das empresas
ordem_empresas = ['AZU', 'GLO', 'IPM', 'ONE', 'PTB', 'TAM']

# Paleta de cores personalizada
cores_empresas = {
    'AZU': "#0015ce",
    'GLO': "#fb8e00",
    'IPM': "#766363",
    'ONE': '#2ca02c',
    'PTB': '#9467bd',
    'TAM': "#ff0000"
}

grafico_evolucao_ms = (
    alt.Chart(df_ms)
    .mark_bar()
    .encode(
        x=alt.X('ANO:O', title='Ano'),
        y=alt.Y('PERCENTUAL:Q', title='Market Share', axis=alt.Axis(format='.0%')),
        color=alt.Color(
            'EMPRESA:N',
            scale=alt.Scale(domain=ordem_empresas, range=[cores_empresas[e] for e in ordem_empresas]),
            title='Empresa'
        ),
        order=alt.Order('EMPRESA:N', sort='ascending'),
        tooltip=['ANO', 'EMPRESA', alt.Tooltip('PERCENTUAL:Q', format='.1%')]
    )
    .properties(
        width=600,
        height=400,
        title='Market Share por Empresa - Coluna Empilhada'
    )
)

grafico_evolucao_ms

alt.Chart(...)

In [20]:
'''
Este gráfico ilustra a evolução da participação de mercado (Market Share) das companhias aéreas na rota Rio-SP entre 2016 e 2024.
É possível identificar movimentos estruturais importantes no setor:

A Saída da Avianca (ONE): Observa-se claramente o impacto da recuperação judicial e falência da Avianca Brasil em 2019.
Até 2018, a empresa detinha uma fatia relevante (~20%), que foi absorvida pelas concorrentes nos anos seguintes.

Consolidação em Oligopólio: Após 2019, o mercado se consolidou em torno de três grandes players: LATAM, GOL e AZUL.

Disputa pela Liderança: A GOL historicamente mantém a maior fatia nesta rota (chegando a quase 50% em 2021), 
mas nota-se um crescimento consistente da AZUL, que passou de cerca de 10% em 2018 para patamares acima de 30% em 2023/2024, 
acirrando a concorrência.
'''

'\nEste gráfico ilustra a evolução da participação de mercado (Market Share) das companhias aéreas na rota Rio-SP entre 2016 e 2024.\nÉ possível identificar movimentos estruturais importantes no setor:\n\nA Saída da Avianca (ONE): Observa-se claramente o impacto da recuperação judicial e falência da Avianca Brasil em 2019.\nAté 2018, a empresa detinha uma fatia relevante (~20%), que foi absorvida pelas concorrentes nos anos seguintes.\n\nConsolidação em Oligopólio: Após 2019, o mercado se consolidou em torno de três grandes players: LATAM, GOL e AZUL.\n\nDisputa pela Liderança: A GOL historicamente mantém a maior fatia nesta rota (chegando a quase 50% em 2021), \nmas nota-se um crescimento consistente da AZUL, que passou de cerca de 10% em 2018 para patamares acima de 30% em 2023/2024, \nacirrando a concorrência.\n'

In [21]:
dfrota
df_tarifa_media = (
    dfrota
    .groupby(['ANO', 'EMPRESA'])
    .agg(
        RECEITA_TOTAL=('RECEITA', 'sum'),
        ASSENTOS_TOTAL=('ASSENTOS', 'sum')
    )
    .reset_index()
)

# Tarifa média anual
df_tarifa_media['TARIFA_MEDIA'] = df_tarifa_media['RECEITA_TOTAL'] / df_tarifa_media['ASSENTOS_TOTAL']

df_tarifa_media


,ANO,EMPRESA,RECEITA_TOTAL,ASSENTOS_TOTAL,TARIFA_MEDIA
0,2016,AZU,9.511267e+07,308443,308.363843
1,2016,GLO,1.666828e+08,771457,216.062348
2,2016,ONE,1.784912e+08,638668,279.474165
3,2016,TAM,1.789461e+08,620996,288.159853
4,2017,AZU,8.604188e+07,280411,306.842026
5,2017,GLO,1.897112e+08,793991,238.933684
6,2017,ONE,1.464797e+08,516472,283.615954
7,2017,TAM,1.323174e+08,546151,242.272619
8,2018,AZU,7.525583e+07,197061,381.891038
9,2018,GLO,2.067271e+08,839398,246.280248


In [22]:
#----------------------------------------- Gráfico de dispersão ---------------------------------------------------------------

df_tarmedia_marketshare = pd.merge(
    df_tarifa_media[['ANO', 'EMPRESA', 'TARIFA_MEDIA']],
    df_ms[['ANO', 'EMPRESA', 'PERCENTUAL']],
    on=['ANO', 'EMPRESA'],
    how='inner'
)

cores_empresas = {
    'AZU': "#0015ce",
    'GLO': "#fb8e00",
    'IPM': "#766363",
    'ONE': '#2ca02c',
    'PTB': '#9467bd',
    'TAM': "#ff0000"
}

correlacao_ms = df_tarmedia_marketshare['PERCENTUAL'].corr(df_tarmedia_marketshare['TARIFA_MEDIA'])
grafico_dispercao = (
    alt.Chart(df_tarmedia_marketshare)
    .mark_circle(size=150)
    .encode(
        x=alt.X(
            'PERCENTUAL:Q',
            title='Market Share (%)',
            axis=alt.Axis(format='%', grid=False)
        ),
        y=alt.Y(
            'TARIFA_MEDIA:Q',
            title='Tarifa Média (R$)',
            axis=alt.Axis(grid=False)
        ),
        color=alt.Color('EMPRESA:N',
        scale=alt.Scale(domain=ordem_empresas, range=[cores_empresas[e] for e in ordem_empresas]),
        title='Empresa'
        ),
        shape='ANO:N',
        tooltip=[
            'ANO:N',
            'EMPRESA:N',
            alt.Tooltip('PERCENTUAL:Q', format='.1%'),
            alt.Tooltip('TARIFA_MEDIA:Q', format='.2f')
        ]
    )
    .properties(title=f'Relação entre Market Share e Tarifa Média (correlação {correlacao_ms:.2f}) ',  width=600,
    height=400) 
)


#Apresenta a relação de Market Share e Tarifa Média
grafico_dispercao


alt.Chart(...)

In [23]:
'''
Este gráfico de dispersão relaciona a fatia de mercado de cada empresa em um determinado ano (eixo X)
com a tarifa média praticada por ela (eixo Y).

Correlação Negativa (-0.43): O índice de correlação indica uma tendência inversa moderada.
Em geral, quanto maior o Market Share da empresa, menor tende a ser a tarifa média cobrada.

Isso sugere que as empresas líderes conseguem praticar preços mais competitivos,
possivelmente devido a economias de escala (diluição de custos fixos por um maior volume de passageiros) ou como
estratégia agressiva para manutenção da sua posição dominante. Por outro lado, empresas com menor participação muitas
vezes operam com custos unitários mais altos ou focam em nichos específicos, resultando em tarifas médias superiores.
'''

'\nEste gráfico de dispersão relaciona a fatia de mercado de cada empresa em um determinado ano (eixo X)\ncom a tarifa média praticada por ela (eixo Y).\n\nCorrelação Negativa (-0.43): O índice de correlação indica uma tendência inversa moderada.\nEm geral, quanto maior o Market Share da empresa, menor tende a ser a tarifa média cobrada.\n\nIsso sugere que as empresas líderes conseguem praticar preços mais competitivos,\npossivelmente devido a economias de escala (diluição de custos fixos por um maior volume de passageiros) ou como\nestratégia agressiva para manutenção da sua posição dominante. Por outro lado, empresas com menor participação muitas\nvezes operam com custos unitários mais altos ou focam em nichos específicos, resultando em tarifas médias superiores.\n'

In [24]:


dash = alt.hconcat(
alt.vconcat(dispersao_tarifa_assentos_sem20e21, grafico_final_heatmap),
alt.vconcat(dispersao_QAV_final, grafico_dispercao)
)


dash

alt.HConcatChart(...)